In [ ]:
import numpy as np
import scipy.integrate as itg
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
# load data
data_5 = np.loadtxt('data/physical_adsorption/241030-4A-5.csv', delimiter=',', skiprows=1).T
data_10 = np.loadtxt('data/physical_adsorption/241030-4A-10.csv', delimiter=',', skiprows=1).T
data_15 = np.loadtxt('data/physical_adsorption/241030-4A-15.csv', delimiter=',', skiprows=1).T
data_20 = np.loadtxt('data/physical_adsorption/241030-4A-20.csv', delimiter=',', skiprows=1).T

data = [data_5, data_10, data_15, data_20]

start_end = np.loadtxt('data/physical_adsorption/start_end.csv', delimiter=',')

In [ ]:
plt.figure(figsize=(10, 8))  # 设置图形大小

# 创建一个格式化器对象，设置使用科学记数法（指数）
formatter = ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((-1, 1))  # 设置科学记数法阈值

# 第一个子图
ax1 = plt.subplot(2, 2, 1)
ax1.plot(data_5[0], data_5[2], label='Rate of N2: 5')
ax1.set_title('Rate of N2: 5')
ax1.set_xlabel('Time')
ax1.set_ylabel('Value')
ax1.legend(loc='upper left')
ax1.xaxis.set_major_formatter(formatter)  # 设置x轴格式器

# 第二个子图
ax2 = plt.subplot(2, 2, 2)
ax2.plot(data_10[0], data_10[2], label='Rate of N2: 10')
ax2.set_title('Rate of N2: 10')
ax2.set_xlabel('Time')
ax2.set_ylabel('Value')
ax2.legend(loc='upper left')
ax2.xaxis.set_major_formatter(formatter)

# 第三个子图
ax3 = plt.subplot(2, 2, 3)
ax3.plot(data_15[0], data_15[2], label='Rate of N2: 15')
ax3.set_title('Rate of N2: 15')
ax3.set_xlabel('Time')
ax3.set_ylabel('Value')
ax3.legend(loc='upper left')
ax3.xaxis.set_major_formatter(formatter)

# 第四个子图
ax4 = plt.subplot(2, 2, 4)
ax4.plot(data_20[0], data_20[2], label='Rate of N2: 20')
ax4.set_title('Rate of N2: 20')
ax4.set_xlabel('Time')
ax4.set_ylabel('Value')
ax4.legend(loc='upper left')
ax4.xaxis.set_major_formatter(formatter)

# 调整子图间距，避免标签重叠
plt.tight_layout()

plt.show()

In [ ]:
# claculate the area for data_5
_idx = 0
_class = 0
result = []
_result = []
for ab in start_end:
  if _idx > 2:
    _class += 1
    _idx = 0
    result.append(_result)
    _result = []

  _data = data[_class]

  a, b = ab
  start_ = np.searchsorted(_data[0], a)
  end_ = np.searchsorted(_data[0], b)

  area = itg.simpson(_data[2][start_ : end_ + 1], x=_data[0][start_ : end_ + 1])
  # area = itg.trapezoid(_data[2][start_ : end_ + 1], x=_data[0][start_ : end_ + 1])

  _result.append(area)

  print(f'data_{(_class + 1)*5}  {a} to {b}: {area:e}')
  _idx += 1

result.append(_result)

with open('data/physical_adsorption/area.csv', 'w') as f:
  for r in result:
    f.write(','.join([str(i) for i in r]) + '\n')

In [ ]:
V_std = 1.05  # standard volume of N2, mL

result_T = np.array(result).T

adsorption_area = np.array(result_T[0])
print(f'adsorption_area: {adsorption_area}')
desorption_area = np.array(result_T[1])
print(f'desorption_area: {desorption_area}')
std_area = np.array(result_T[2])
print(f'std_area: {std_area}')

adsorption_volume = adsorption_area / std_area * V_std  # mL
desorption_volume = desorption_area / std_area * V_std  # mL

print(f'adsorption_volume: {adsorption_volume}')
print(f'desorption_volume: {desorption_volume}')

In [ ]:
P_std = 102000  # standard pressure, Pa
velocity_N2 = np.array([5, 10, 15, 20])
velocity_He = 40
z = velocity_N2 / (velocity_N2 + velocity_He)  # unitless

print(f'z: {z}')

y = z / (1 - z) / desorption_volume

print(f'desorption_volume: {desorption_volume}')

print(f'y: {y}')

slope, intercept, r_value, p_value, std_err = st.linregress(z, y)

single_vm = []
print('=====================')
for i in range(len(z)):
  print(f'z: {z[i]}  y: {y[i]}')
  single_vm.append(z[i] / y[i])
print('=====================')
print(f'single_vm: {single_vm}')

print(f'slope: {slope}')
print(f'intercept: {intercept}')
print(f'r2: {r_value**2}')

Vm = 1 / (intercept + slope)  # mL
print(f'Vm: {Vm}')

R = 8.314  # J/(mol*K)
T = 22.4 + 273.15  # K

m = 20.4504 - 20.2620  # g
print(f'm: {m}g')

nm = (P_std * Vm * 1e-6) / (R * T) / m  # mol
print(f'nm: {nm}')

am = 0.162e-18  # m^2
NA = 6.022e23  # mol^-1
S = am * NA * nm  # m^2 g

print(f'S: {S}')

single_nm = np.array(single_vm) / Vm * nm  # mol g^-1
print(f'single_nm: {single_nm}')
single_s = am * NA * single_nm  # m^2 g^-1
print(f'single_s: {single_s}')

# plot the linear regression
_z = np.linspace(0.05, 0.35, 100)
_y = slope * _z + intercept

plt.plot(z, y, 'o', label='data')
plt.plot(_z, _y, '--', label='linear regression')
plt.title('y of z')
plt.xlim(0.05, 0.35)
plt.xlabel('z')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
std_s = 69.1  # m^2 g^-1
std_single_s = 67.8  # m^2 g^-1

error_s = (S - std_s) / std_s * 100
error_single_s = (single_s - std_single_s) / std_single_s * 100

print(f'error_s: {error_s}%')
print(f'error_single_s: {error_single_s}%')